In [1]:
# FedAvg
# FedFair
# data distribution that we used

# Recall
# FedAvg + FairFed + normal model

# Plots withing notbooks
# Plots for Acc vs subGroup
# Show data distrubution
# Recall of positive class
# the data split we use....

In [2]:
import pandas as pd
import time
import numpy as np
import copy
import glob
import sys, os
import torch
import torch.nn as nn
import torch.utils.data as data_utils
from sklearn.metrics import precision_score, recall_score
import pickle
from sklearn import preprocessing
import torch.optim as optim
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from folktables import ACSDataSource, ACSEmployment,ACSIncome
import matplotlib.pyplot as plt
import torch.nn.functional as F
from tqdm import tqdm
from sklearn.metrics import roc_curve
from sklearn.model_selection import StratifiedKFold, train_test_split
from torchvision import models
from torchsummary import summary
from torch.utils.data import Dataset, DataLoader
from sklearn.metrics import accuracy_score
import os
from torch.utils.data import Dataset
import numpy as np
import torch.nn.functional as F
import pandas as pd
from scipy.stats import multivariate_normal
import torch, random, copy, os
from collections import OrderedDict


In [3]:
# TPR
# Gender: 
# 1: Male, 0: Female
# Fairness: Male- Female
# eq-4: Global- TPR

# Fk = 

In [4]:
# !pip install ray

# data preparation

In [5]:
import torch

if torch.cuda.is_available():
    print("CUDA is available.")
else:
    # CUDA is not available
    print("CUDA is not available. Running on CPU.")


CUDA is available.


In [6]:
os.getcwd()

'/home/chiragapandav/Downloads/Hiwi/Improving-Fairness-via-Federated-Learning/FedFB'

In [7]:
 
class DeepNet(nn.Module):
    def __init__(self):
        super().__init__()
        # 14 : input shape
        # 9-> we have 9 columns in data 
        self.layer1 = nn.Linear(9, 512)
        self.act1 = nn.ReLU()
        self.dropout1 = nn.Dropout(p=0.5)
        self.layer2 = nn.Linear(512, 256)
        self.act2 = nn.ReLU()
        self.layer3 = nn.Linear(256, 60)
        self.act3 = nn.ReLU()
        self.output = nn.Linear(60, 1)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        x = self.act1(self.layer1(x))
        x = self.dropout1(x)
        x = self.act2(self.layer2(x))
        x = self.act3(self.layer3(x))
        x = self.sigmoid(self.output(x))
        return x



# assigning Models

In [8]:
#create global model and client models
temp_model = DeepNet()
path="/home/chiragapandav/Downloads/Hiwi/Improving-Fairness-via-Federated-Learning/FedFB/models/global_model_2.pt"
torch.save(temp_model.state_dict(), path)

# this has to be same in follwing code as well
selected_clients=[0,1,2,3,4,5,6,7,8,9]

for client_id in selected_clients:
    client_model_path = f"/home/chiragapandav/Downloads/Hiwi/Improving-Fairness-via-Federated-Learning/FedFB/models/client_{client_id}_model.pth"
    torch.save(temp_model.state_dict(), client_model_path)
    


In [9]:

class Client:
    def __init__(self):
        self.client_id: int = None
        self.valset: DataLoader = None
        self.trainset: DataLoader = None
        self.testset: DataLoader = None
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")         
        self.model = DeepNet().to(self.device)
        self.criterion = torch.nn.BCELoss()

        self.fainess_score=0

        self.info_collection=[]
        self.client_df = pd.DataFrame()

    def get_client_local_dataset(self):
                
        temp_path_data="/home/chiragapandav/Downloads/Hiwi/Improving-Fairness-via-Federated-Learning/FedFB/data_fairFed"
        
        with open(temp_path_data+"/clients_training.pkl", "rb") as f:
            self.trainset = pickle.load(f)
            
        with open(temp_path_data+"/clients_validation.pkl", "rb") as f:
            self.valset = pickle.load(f)
         
        with open(temp_path_data+"/clients_testing.pkl", "rb") as f:
            self.testset  = pickle.load(f)

        self.trainset = self.trainset[self.client_id]
        self.valset = self.valset[self.client_id]       
        self.testset = self.testset[self.client_id]

    def calculate_fairness(self,y_hat, A, Y):
        # Calculate counts using torch.sum
        y_hat, A, Y = y_hat.to(self.device), A.to(self.device), Y.to(self.device)

        # 1: Male, 0: Female
        predict_Y_male = torch.sum((y_hat[(A == 1) & (Y == 1)] == 1)).item()
        predict_Y_female = torch.sum((y_hat[(A == 0) & (Y == 1)] == 1)).item()
        
        count_Y_male = torch.sum((Y[(A == 1) & (Y == 1)] == 1)).item()
        count_Y_female = torch.sum((Y[(A == 0) & (Y == 1)] == 1)).item()
    
        # Calculate probabilities
        prob_Y_male = predict_Y_male / count_Y_male if count_Y_male > 0 else 0
        prob_Y_female = predict_Y_female / count_Y_female if count_Y_female > 0 else 0
    
        # Calculate Fglobal
        fairness_score = prob_Y_male - prob_Y_female
        
        return  fairness_score

    def train(self, client_id: int, model_params: OrderedDict[str, torch.Tensor], num_epochs=5, learning_rate=0.001):
        self.client_id = client_id
        self.get_client_local_dataset()

        # Define loss function and optimizer
        criterion = nn.BCELoss()
        optimizer = optim.Adam(self.model.parameters(), lr=learning_rate)
        self.fainess_score=0

        for epoch in range(num_epochs):
            # Set the model to training mode
            self.model.train()

            for inputs, labels, sens in self.trainset:
                
                inputs, labels = inputs.to(self.device), labels.to(self.device)
                optimizer.zero_grad()
                outputs = self.model(inputs)
                loss = self.criterion(outputs, labels.float())
                loss.backward()
                optimizer.step()

            # Calculate and print the training accuracy for this epoch (optional)
            correct = 0
            total = 0
            size = 0
            loss = 0
            correct = 0
            total = 0
            predicted_labels = []
            true_labels = []
            final_fairness=[]
            
            with torch.no_grad():
                print("======Validation========")
                for inputs, labels,sens in self.valset:
                    inputs, labels = inputs.to(self.device), labels.to(self.device)
                    
                    outputs = self.model(inputs)
                    loss += self.criterion(outputs, labels)
                    predicted = outputs > 0.5

                    predicted_labels.extend(predicted.cpu().numpy())
                    true_labels.extend(labels.cpu().numpy())

                    fairNess_per_batch=self.calculate_fairness(torch.round(outputs).squeeze(), sens.squeeze(), labels.squeeze())
                    final_fairness.append(fairNess_per_batch)
                    self.fainess_score=np.mean(final_fairness)

                    total += labels.size(0)
                    correct += (predicted == labels).sum().item()

            loss = loss / len(self.valset)        
            accuracy = correct / total
            
            # accuracy = 100 * correct / total
            # print(f"Epoch {epoch+1}/{num_epochs}, Training Accuracy: {accuracy:.5f}%")
            print(f"Epoch {epoch+1}/{num_epochs}, Training Fairness: {self.fainess_score:.5f}%")

        # Optionally, save the trained model parameters
        client_model_path = f"/home/chiragapandav/Downloads/Hiwi/Improving-Fairness-via-Federated-Learning/FedFB/models/client_{self.client_id}_model.pth"
        torch.save(self.model.state_dict(), client_model_path)

        # Return the trained model parameters
        return list(self.model.state_dict().values()), len(self.trainset.dataset),self.model.state_dict(), self.fainess_score

    @torch.no_grad()
    def client_evaluate(self, val=False):
        
        size = 0
        loss = 0
        correct = 0
        total = 0
        predicted_labels = []
        true_labels = []

        selected_clients=[0,1]
        models_directory = "/home/chiragapandav/Downloads/Hiwi/Improving-Fairness-via-Federated-Learning/FedFB/models/"
                 
        for client_id in selected_clients:
            
            print("===============Testing======================")
            # print(" selected client for Testing:: ", client_id)
            model_path = os.path.join(models_directory, f"client_{client_id}_model.pth")            
        
            self.model.load_state_dict(torch.load(model_path,map_location=self.device))

            self.model.eval()

            for inputs, targets,sens in self.testset:
                
                inputs, targets = inputs.to(self.device), targets.to(self.device)
                
                outputs = self.model(inputs)
                loss += self.criterion(outputs, targets)
                predicted = outputs > 0.5
                
                predicted_labels.extend(predicted.cpu().numpy())
                true_labels.extend(targets.cpu().numpy())
                
                total += targets.size(0)
                correct += (predicted == targets).sum().item()
                
            
            loss = loss / len(self.testset)
            acc = correct / total
            
            # print("loss: %f\n" % (loss))
            
            
            print(f" Client Accuracy: {acc:.5%}")
    #         print(predicted_labels," :: ",true_labels)
            precision = precision_score(true_labels, predicted_labels,zero_division=0.0)
            
            recall = recall_score(true_labels, predicted_labels)
    
            print(f" Client Precision: {precision:.5%}")
            print(f" Client Recall: {recall:.5%}")


            # client_round_info = {        
            
            # "Data Size": len(self.testset),  # Update this with the actual data size
            # "Accuracy": acc,
            # "Recall": recall,
            # "Precision": precision
            # }
            # # self.client_df = self.client_df.concat(client_round_info, ignore_index=True)
            
            # self.client_df =pd.concat(self.client_df,pd.DataFrame([client_round_info]))
            # client_df.to_csv("hello.csv", index=False)
 # df = pd.concat([df, pd.DataFrame([result])])
            print("\n\n")
            
        return loss, acc

    
    @torch.no_grad()
    def server_evaluate(self):

        # print("Global Model testing Starts")
        # print("kindly check the Path. Select it based on FedAvg Model")
        # print("warning: Test data has already been used")
        
        path="/home/chiragapandav/Downloads/Hiwi/Improving-Fairness-via-Federated-Learning/FedFB/models/global_model.pt"
        path_2="/home/chiragapandav/Downloads/Hiwi/Improving-Fairness-via-Federated-Learning/FedFB/models/global_model_2.pt"
        
        self.model.load_state_dict(torch.load(path_2,map_location=self.device))
        self.model.eval()
        size = 0
        loss = 0
        correct = 0
        total = 0
        predicted_labels = []
        true_labels = []
        final_fairness=[]
        for inputs, targets,sens in self.testset:
            
            inputs, targets = inputs.to(self.device), targets.to(self.device)
            
            outputs = self.model(inputs)
            loss += self.criterion(outputs, targets)
            predicted = outputs > 0.5

            fairNess_per_batch=self.calculate_fairness(torch.round(outputs).squeeze(), sens.squeeze(), targets.squeeze())
            final_fairness.append(fairNess_per_batch)
            self.fainess_score_global=np.mean(final_fairness)
            
#            print("predicted",predicted)
            predicted_labels.extend(predicted.cpu().numpy())
            true_labels.extend(targets.cpu().numpy())
            
            total += targets.size(0)
            correct += (predicted == targets).sum().item()
            
        
        loss = loss / len(self.testset)
        acc = correct / total
        
        # print("loss: %f\n" % (loss))
        
        print(f"Global Testing Accuracy: {acc:.5%}")
        print(f" Global Fairness: {self.fainess_score_global:.5f}%")
        precision = precision_score(true_labels, predicted_labels,zero_division=0.0)
        
        recall = recall_score(true_labels, predicted_labels)
        
        print(f"Global Precision: {precision:.5%}")
        print(f"Global Recall: {recall:.5%}")
        
        return loss, acc, self.fainess_score_global
                                            

In [10]:
class Serverbase:
    def __init__ (self,model):
        self.model = model
        self.global_model=model
        self.num_rounds=5
        self.local_epoch=10
        self.optimizer=2
        self.lr=0.001
        self.beta=0.02
        self.batch_size=32
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu") 

        self.client_id_indices, self.client_num_in_total = [0,1], 2 
        
        self.updated_params_cache = []
        self.weights_cache = []
        self.model_dict = []
        
        self.global_params_dict: OrderedDict[str : torch.Tensor] = None

        self.backbone=DeepNet
        _dummy_model = self.backbone()
        self.global_params_dict = OrderedDict(_dummy_model.state_dict())

        self.temp_dir="/home/chiragapandav/Downloads/Hiwi/Improving-Fairness-via-Federated-Learning/FedFB/models/"

        self.fair_global_t_step=0
        self.fair_local_client=0   
        self.acc_clients=[]
        self.global_acc=0
        self.fairness_diff=[]
        self.clients_weights=[]
        self.final_agg_weights_clients=[]
        self.selected_clients=[0,1,2,3,4,5,6,7,8,9]
        
    def fairFed(self, num_rounds = 2, local_epochs = 2, learning_rate = 0.005, beta = 0.3, alpha = 0.1, optimizer = 'adam'):
        
        for round_ in tqdm(range(num_rounds)):
  
            #each round we need empty lists
            self.updated_params_cache = []
            self.weights_cache = []
            self.model_dict=[]
            self.clients_weights=[]
            self.final_agg_weights_clients=[]
            
            for client_id in self.selected_clients:
                self.global_acc=[]
                client = Client()
                print("client_id:: ",client_id)
                # weight is length of dataset
                updated_params_list, weight, model_dict_list,fairness_client = client.train(client_id, self.model.state_dict(), num_epochs=local_epochs, learning_rate=learning_rate)
                loss_client, acc_client = client.client_evaluate(val=True)
                loss_server, acc_server,fairness_global = client.server_evaluate()
                
                # store it for SecAgg
                self.updated_params_cache.append(updated_params_list)
                self.weights_cache.append(weight)
                self.model_dict.append(model_dict_list)
                
                #store it for FairFed agg
                self.acc_clients.append(acc_client)
                self.global_acc=acc_server
                self.fair_local_client=fairness_client
                self.fair_global=fairness_global
        
            # self.aggregate_parameters_FedAvg(self.updated_params_cache, self.weights_cache)
            
            self.aggregate_client_weight_FedFair_part_1(self.model_dict, self.weights_cache,self.acc_clients, self.global_acc,self.fair_local_client,
                                                 self.fair_global)
            self.client_weights_update_part_2(self.clients_weights)
            # print("self.model_dict",self.model_dict)
            self.aggregate_parameters_FedAvg_updated(self.final_agg_weights_clients)
                
            torch.save(self.global_params_dict, os.path.join(self.temp_dir, "global_model.pt"),)

        # Test Clients Models and Global Model
        # loss_client, acc_client = client.client_evaluate()        
        # loss_server, acc_server = client.server_evaluate()
        

    @torch.no_grad()
    def aggregate_parameters_FedAvg(self, updated_params_cache, weights_cache):
        weight_sum = sum(weights_cache)  
        
        weights = torch.tensor(weights_cache, device=self.device) / weight_sum
        
        aggregated_params = []

        for params in zip(*updated_params_cache):
            aggregated_params.append(
                torch.sum(weights * torch.stack(params, dim=-1), dim=-1)
            )

        self.global_params_dict = OrderedDict(
            zip(self.global_params_dict.keys(), aggregated_params)
        )

    
    @torch.no_grad()
    def aggregate_parameters_FedAvg_updated(self, model_dict_list):        
        w_avg = copy.deepcopy(model_dict_list[0])
        for k in w_avg.keys():
            for i in range(1, len(model_dict_list)):
                w_avg[k] += model_dict_list[i][k]
            w_avg[k] = torch.div(w_avg[k], len(model_dict_list))
    
        self.global_model.load_state_dict(w_avg)
        torch.save(self.global_model.state_dict(), os.path.join(self.temp_dir, "global_model_2.pt"),)
            
    def client_weights_update_part_2(self,agg_weights):
        sum_params = OrderedDict()
        final_agg_weights_clients=[]
        for model in agg_weights:
            for name, param in model.items():
                if name in sum_params:
                    sum_params[name] += param
                else:
                    sum_params[name] = param

            updated_client_weights = OrderedDict()
            
            for name, param in model.items():
                updated_param = torch.div(param, sum_params[name])
                updated_client_weights[name] = updated_param

                self.final_agg_weights_clients.append(updated_client_weights)
                
        
    @torch.no_grad()
    def aggregate_client_weight_FedFair_part_1 (self, model_dict_list ,weights_cache, acc_clients, global_acc,fairness_client,fairness_global):
        print("len::",len(model_dict_list))
        
        if not self.fair_local_client:
            
            sum_acc_client=sum(acc_clients) /len(acc_clients)
            Delta_t_C_i= abs(sum_acc_client-global_acc)
            
        else:            
            Delta_t_C_i = abs(self.fair_global - self.fair_local_client) 
        
        for client_id in range(len(self.selected_clients)):
        
            w_avg = copy.deepcopy(model_dict_list[client_id])
            temp = copy.deepcopy(model_dict_list[client_id])
            
            for k in w_avg.keys():
                for i in range(1, len(model_dict_list)):
                    w_avg[k] += model_dict_list[i][k]
                w_avg[k] = torch.div(w_avg[k], len(model_dict_list))
                
                w_avg[k]=temp[k]-(self.beta*(Delta_t_C_i-w_avg[k]))
                
            self.clients_weights.append(w_avg)
                        
        
model = DeepNet()
server = Serverbase(model)
server.fairFed(num_rounds=2,local_epochs=10)    

  0%|                                                                                                                        | 0/2 [00:00<?, ?it/s]

client_id::  0
======Validation========
Epoch 1/10, Training Fairness: -0.01884%
======Validation========
Epoch 2/10, Training Fairness: 0.06187%
======Validation========
Epoch 3/10, Training Fairness: -0.06051%
======Validation========
Epoch 4/10, Training Fairness: -0.05847%
======Validation========
Epoch 5/10, Training Fairness: -0.10596%
======Validation========
Epoch 6/10, Training Fairness: -0.10596%
======Validation========
Epoch 7/10, Training Fairness: -0.09586%
======Validation========
Epoch 8/10, Training Fairness: -0.06808%
======Validation========
Epoch 9/10, Training Fairness: -0.00923%
======Validation========
Epoch 10/10, Training Fairness: -0.09897%
===============Testing======================
 Client Accuracy: 79.21225%
 Client Precision: 66.82243%
 Client Recall: 85.62874%



===============Testing======================
 Client Accuracy: 66.30197%
 Client Precision: 52.58964%
 Client Recall: 79.04192%



Global Testing Accuracy: 53.39168%
 Global Fairness: -0.04941%


 50%|████████████████████████████████████████████████████████                                                        | 1/2 [00:13<00:13, 13.52s/it]

======Validation========
Epoch 10/10, Training Fairness: 0.10979%
===============Testing======================
 Client Accuracy: 75.82090%
 Client Precision: 68.84735%
 Client Recall: 78.09187%



===============Testing======================
 Client Accuracy: 76.04478%
 Client Precision: 68.87519%
 Client Recall: 78.97527%



Global Testing Accuracy: 57.46269%
 Global Fairness: 0.06276%
Global Precision: 49.75845%
Global Recall: 72.79152%
len:: 10
client_id::  0
======Validation========
Epoch 1/10, Training Fairness: -0.13316%
======Validation========
Epoch 2/10, Training Fairness: -0.02205%
======Validation========
Epoch 3/10, Training Fairness: 0.01962%
======Validation========
Epoch 4/10, Training Fairness: -0.05740%
======Validation========
Epoch 5/10, Training Fairness: -0.14122%
======Validation========
Epoch 6/10, Training Fairness: -0.03594%
======Validation========
Epoch 7/10, Training Fairness: -0.10276%
======Validation========
Epoch 8/10, Training Fairness: -0.23145%
======

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:26<00:00, 13.16s/it]

======Validation========
Epoch 10/10, Training Fairness: 0.04101%
===============Testing======================
 Client Accuracy: 74.17910%
 Client Precision: 67.97386%
 Client Recall: 73.49823%



===============Testing======================
 Client Accuracy: 75.52239%
 Client Precision: 69.38111%
 Client Recall: 75.26502%



Global Testing Accuracy: 42.23881%
 Global Fairness: 0.00000%
Global Precision: 42.23881%
Global Recall: 100.00000%
len:: 10


In [11]:
# loss_after, acc_after = server.server_evaluate()

In [12]:
# def FedAvg(w, clients):
        #lenghth of data is clients
#     print(clients)
#     w_avg = copy.deepcopy(w[0])
#     for k in w_avg.keys():
#         for i in range(1, len(w)):
#             tens = torch.mul(w[i][k], clients[i])
#             w_avg[k] += tens
#         w_avg[k] = torch.div(w_avg[k], sum(clients))
#     return w_avg

In [13]:
# def FedAvg(w):
#     w_avg = copy.deepcopy(w[0])
#     for k in w_avg.keys():
#         for i in range(1, len(w)):
#             w_avg[k] += w[i][k]
#         w_avg[k] = torch.div(w_avg[k], len(w))
#     return w_avg